In [3]:
flask_code = """from flask import Flask, render_template, request, send_file
import qutip as qt
import matplotlib.pyplot as plt
import io

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route('/generate_circuit', methods=['POST'])
def generate_circuit():
    data = request.json
    qubit_number = int(data['qubitNumber'])
    layers = data['layers']

    # Initialize quantum circuit
    circuit = qt.Qobj(qt.qeye([2] * qubit_number))

    for layer in layers:
        gate = layer['gate']
        qubits = list(map(int, layer['qubit'].split(',')))

        if gate == 'X':
            op = qt.sigmax()
        elif gate == 'Y':
            op = qt.sigmay()
        elif gate == 'Z':
            op = qt.sigmaz()
        elif gate == 'H':
            op = qt.hadamard_transform()
        elif gate == 'CNOT':
            op = qt.cnot(N=qubit_number, control=qubits[0], target=qubits[1])
            circuit = op * circuit
            continue

        for qubit in qubits:
            op_list = [qt.qeye(2)] * qubit_number
            op_list[qubit - 1] = op
            op_full = qt.tensor(op_list)
            circuit = op_full * circuit

    # Plot the circuit
    fig, ax = plt.subplots()
    qt.matrix_histogram_complex(circuit)
    buf = io.BytesIO()
    plt.savefig(buf, format='jpg')
    buf.seek(0)
    plt.close()

    return send_file(buf, mimetype='image/jpeg')

if __name__ == '__main__':
    app.run(debug=True)
"""

# Save the file
with open("app.py", "w") as f:
    f.write(flask_code)

print("app.py saved successfully!")



app.py saved successfully!
